In [ ]:
# Some of this code could be incorporated into the fine-tuning code for simplicity

# import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, set_seed, TrainingArguments, Trainer
import transformers
!pip install datasets
!pip install mlflow
from datasets import load_dataset, Features, Value, Sequence
from pathlib import Path
import mlflow
import torch
import os
import json
!pip install pynvml

In [ ]:
# Load fine-tuned model

fine_tuned_model = AutoModelForCausalLM.from_pretrained(
    'models/olmo_instruct_gpac/',
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

In [ ]:
# Create function to generate assessments of experimental data on one metric described in the "content" below

def generate(prompt, max_new_tokens=1000, chat=True):
    messages = [
        {
            "role": "system",
            "content": """You are a grant review panelist. Score the following application on a scale of 0-35. 0 being the lowest and 35 being the highest. Provide a brief explanation of your score based on the following criteria:
                        The work deepens its impact by welcoming people of all abilities and backgrounds.
                        Strong ideas expressed with clarity advance artistic goals.
                        The emotional and sensory impact of the work engages community.
                        The creative work demonstrates integrity and ethical use of material with specific cultural origins and context.
                        A work can disrupt artistic conventions, concepts of beauty, or how the audience engages. Effective disruption is mindful and intentional.
                        """
        },
        {"role": "user", "content": prompt},
    ]

    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

    input_ids = tokenizer(formatted_prompt, return_tensors="pt").input_ids.to(
        fine_tuned_model.device
    )

    gen_tokens = fine_tuned_model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1,
    )
    return tokenizer.batch_decode(gen_tokens, skip_special_tokens=False)[0]

In [ ]:
# Base code to run across all generated data
# For simplicity and illustrative purposes this code is specifically assessing one experimental application
# Using a loop over the generator to assess all experimental data provides the necessary systematic
# approach to assessing differences by BIPOC and size

# The below prompt is a fictional museum who application content is unique to the organization but similar in style
# to actual applicant narratives.

promptText = """The American Museum of Traveling Salespeople is dedicated to preserving, celebrating, and exploring the impact of traveling salespeople on American culture, commerce, and storytelling. Through engaging exhibitions and interactive displays, we highlight the ingenuity, resilience, and personal histories of those who built industries from the road. Our vision is to present compelling, innovative exhibits that connect past and present, inspiring reflection on the evolution of sales, entrepreneurship, and human connection.
                We define a "traveling salesperson" as more than a profession—it’s a symbol of adaptability, persuasion, and the pioneering spirit that shaped the American economy. Our curatorial approach treats sales as a cultural narrative, showcasing artifacts, personal accounts, and multimedia presentations that reveal the deep influence of these individuals on business, marketing, and social mobility.
                Our exhibitions bring history to life, but our mission extends beyond preservation. The American Museum of Traveling Salespeople fosters collaboration with historians, business leaders, and storytellers, ensuring that these legacies are studied, understood, and appreciated by future generations.
                The Routes & Roads Project expands our work by uncovering and sharing overlooked stories—from the door-to-door peddlers of the early 20th century to the traveling tech sales reps of today. This year’s theme, "Pitches, Promises, and Persistence," explores the tools, tactics, and tales of those who sold everything from encyclopedias to vacuum cleaners, miracle cures to life insurance. Each exhibit invites visitors to reflect on how sales shaped American innovation and the enduring legacy of those who made their living on the move."""

print(generate(promptText, chat=True))